# Modules

In [7]:
import findspark
findspark.init()

In [8]:
import os
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
import configparser
import realstate_utils as rs


In [9]:
config = configparser.ConfigParser()
config.read('config.ini')

path = config.get('PATH', 'PATH_DATA')
url = config.get('URL', 'URL')
page = int(config.get('PAGE', 'page'))

# Processing

In [3]:
spark = pyspark.sql.SparkSession.builder \
    .appName("Real_state") \
    .getOrCreate()

In [7]:
spark.active()

In [4]:
print(url)

https://www.citya.com/annonces/vente/appartement,maison?sort=b.dateMandat&direction=desc


In [10]:
data = rs.scrapping(url,page)

In [6]:
# Définissez le schéma du DataFrame
schema = StructType([
    StructField("id", StringType(), True),
    StructField("title", StringType(), True),
    StructField("surface", FloatType(), True),
    StructField("price", IntegerType(), True),
    StructField("city", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("number_pieces", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("description", StringType(), True)
    
])
df = spark.createDataFrame(data,schema)

ConnectionRefusedError: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée

In [8]:
df.count()

5421

In [9]:
df.show(1,False)

+---+-----------------------------------------+-------+------+------------+-----------+-------------+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
df1=df\
.withColumn("Id", monotonically_increasing_id())

In [12]:
df_final= df1.select("Id","type","city","postal_code","number_pieces","surface","price")


In [13]:
df_final.show()

+---+-----------+--------------------+-----------+-------------+-------+------+
| Id|       type|                city|postal_code|number_pieces|surface| price|
+---+-----------+--------------------+-----------+-------------+-------+------+
|  0|     Maison|        Francheville|      69340|            7|  193.0|860000|
|  1|Appartement|             Le Mans|      72000|            4|  87.83|250000|
|  2|Appartement|Paris e Arrondiss...|      75019|            3|   60.0|523000|
|  3|     Maison|                 Pau|      64000|            4|  80.59|200000|
|  4|Appartement|                 Pau|      64000|            4|   78.0|132000|
|  5|Appartement|            Poitiers|      86000|            4|  80.75|147000|
|  6|     Maison|              Bonzac|      33910|            6|  157.0|339000|
|  7|     Maison|         Châteauroux|      36000|            5|  108.0|188200|
|  8|Appartement|            Le Havre|      76600|            3|   72.0|430500|
|  9|Appartement|Les Abrets en Dau...|  

In [14]:
df_final.write.parquet("../output/realstate.parquet")

Py4JJavaError: An error occurred while calling o61.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: Hadoop bin directory does not exist: C:\hadoop\bin -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache

: 